In [1]:
import warnings
import pprint
import skrebate
import imblearn
import random
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, LeaveOneGroupOut, cross_validate, cross_val_predict
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.preprocessing import QuantileTransformer, quantile_transform, RobustScaler
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')



/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #620
  (fname, cnt))


In [2]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
combined_master = pd.read_csv('./mIHC/test1_newLabels/test1&3_master_ready.tsv.gz', '\t', index_col=0)
combined_master

,test_no,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
0,test1,4381R6_53753,4381R6,F,0.111675,0.171717,0.108491,0.273196,0.133333,0.136150,...,0.001208,0.009804,0.015385,0.010050,1385.771930,3962.368421,0.000000,171,0.333333,0.151042
1,test3,SMA37F_32367,SMA37F,G,0.013072,0.000000,0.000000,0.000000,0.000000,0.005682,...,0.000000,0.000000,0.000000,0.000000,1070.000000,4240.500000,0.000000,8,0.000000,0.000000
2,test1,4411R4_18331,4411R4,B,0.015504,1.369127,1.126506,0.344538,0.025641,0.945946,...,0.065574,0.000000,0.000000,0.118421,2172.116541,1882.721805,0.000000,266,0.201835,0.006024
3,test3,SMA54F_1539,SMA54F,G,0.086957,0.111111,0.368932,0.173077,0.004505,0.000000,...,0.000000,0.014085,0.014493,0.000000,984.500000,223.500000,0.009709,12,0.150000,0.000000
4,test1,4413R1_2105,4413R1,F,0.202532,0.180723,0.246753,0.234043,0.203704,0.032680,...,0.000000,0.000000,0.012658,0.000000,188.007194,479.100719,0.550000,139,0.392157,0.368098
5,test1,4380R6_13845,4380R6,F,0.710059,0.645714,0.584158,0.851190,0.320513,0.931937,...,0.029268,0.036667,0.005952,0.015228,1156.379121,1775.186813,1.335135,182,1.302158,0.745562
6,test1,4381R6_46009,4381R6,D,0.000000,0.000000,5.528302,0.231959,0.010256,4.004695,...,0.000000,0.000000,0.000000,0.000000,2527.220472,3408.422572,0.000000,381,0.020202,0.000000
7,test1,4547R3_1044,4547R3,F,0.000000,0.035714,0.034091,0.035211,0.010526,0.004525,...,0.000000,0.000000,0.000000,0.000000,3141.260563,220.943662,0.068376,142,0.015152,0.000000
8,test3,SMA40F_19470,SMA40F,A,0.145833,1.496774,2.045161,0.007353,0.000000,0.093897,...,0.062500,0.034884,0.044444,0.000000,939.629870,2876.051948,0.744186,154,0.438356,0.000000
9,test3,SMA60F_720,SMA60F,E,0.152074,0.643243,12.910526,1.536585,0.254464,0.410891,...,0.012658,0.123457,0.013245,0.025974,537.804167,129.704167,0.413534,240,2.277778,3.552000


In [4]:
combined_master['class'].value_counts()

F    10150
E     9255
G     9058
B     7843
H     7118
A     6434
D     5275
C     3150
Name: class, dtype: int64

In [5]:
predicted = pd.read_csv('./Galaxy11-[Model_Validation_on_data_9_and_data_6].tabular', sep='\t')
predicted

,Predicted
0,F
1,G
2,B
3,G
4,F
5,F
6,D
7,F
8,A
9,E


In [6]:
result_combo = combined_master.iloc[:, :4]
result_combo['Predicted'] = predicted['Predicted']
result_combo

,test_no,obj_ID,patient_ID,class,Predicted
0,test1,4381R6_53753,4381R6,F,F
1,test3,SMA37F_32367,SMA37F,G,G
2,test1,4411R4_18331,4411R4,B,B
3,test3,SMA54F_1539,SMA54F,G,G
4,test1,4413R1_2105,4413R1,F,F
5,test1,4380R6_13845,4380R6,F,F
6,test1,4381R6_46009,4381R6,D,D
7,test1,4547R3_1044,4547R3,F,F
8,test3,SMA40F_19470,SMA40F,A,A
9,test3,SMA60F_720,SMA60F,E,E


## How the model performs on each class overall?

In [7]:
# balanced accuracy overall
b_accuracy = balanced_accuracy_score(result_combo['class'], result_combo['Predicted'])
b_accuracy

0.9465286010503184

In [8]:
# confusion matrix overall
mtrix_all = confusion_matrix(result_combo['class'], result_combo['Predicted'],
                            labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
pd.DataFrame(mtrix_all, columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])

,A,B,C,D,E,F,G,H
0,6186,140,9,16,70,2,7,4
1,142,7361,36,72,199,12,4,17
2,22,24,3061,9,29,3,1,1
3,47,71,12,4965,155,11,13,1
4,187,326,45,198,8359,74,37,29
5,17,34,11,54,261,9424,179,170
6,7,23,2,24,51,235,8635,81
7,10,3,4,11,53,78,24,6935


In [9]:
# precision for each class
precision = np.diag(mtrix_all) / mtrix_all.sum(axis=1).astype(float)
pd.DataFrame([precision], columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])

,A,B,C,D,E,F,G,H
0,0.961455,0.938544,0.971746,0.941232,0.903187,0.928473,0.953301,0.974291


In [10]:
# recall for each class
recall = np.diag(mtrix_all) / mtrix_all.sum(axis=0).astype(float)
pd.DataFrame([recall], columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])

,A,B,C,D,E,F,G,H
0,0.934723,0.9222,0.962579,0.928211,0.910864,0.957821,0.970225,0.958138


## How the model performs on each patient?

In [11]:
accuracies = []
for p, p_group in result_combo.groupby(['patient_ID']):
    b_accuracy = balanced_accuracy_score(p_group['class'], p_group['Predicted'])
    accuracies.append(b_accuracy)
    mtrix = confusion_matrix(p_group['class'], p_group['Predicted'], 
                             labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
    precisions = np.diag(mtrix) / mtrix.sum(axis=1).astype(float)
    recalls = np.diag(mtrix) / mtrix.sum(axis=0).astype(float)
    print("For patient %s:"%p)
    print("\tBalanced_accuracy:%f" %b_accuracy)
    print("\tPrecisions for each class:" )
    print(pd.DataFrame([precisions], columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']))
    print("\n\tRecalls for each class:" )
    print(pd.DataFrame([recalls], columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']))
    print("\n\tConfusion matrix:" )
    print(pd.DataFrame(mtrix, columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']))
    print("\n\n")
print("Mean balanced accuracy: %f" %(sum(accuracies)/float(len(accuracies))))    

For patient 4350R3:
	Balanced_accuracy:0.912579
	Precisions for each class:
          A         B    C    D         E     F         G         H
0  0.771429  0.984496  0.8  0.8  0.971223  0.98  0.995223  0.998261

	Recalls for each class:
          A         B    C    D         E         F         G         H
0  0.947368  0.846667  1.0  1.0  0.918367  0.997093  0.993641  0.989655

	Confusion matrix:
    A    B  C   D    E    F    G    H
0  54   16  0   0    0    0    0    0
1   0  127  0   0    1    0    1    0
2   0    1  4   0    0    0    0    0
3   0    0  0  36    9    0    0    0
4   1    3  0   0  135    0    0    0
5   2    3  0   0    2  686    2    5
6   0    0  0   0    0    2  625    1
7   0    0  0   0    0    0    1  574



For patient 4380R6:
	Balanced_accuracy:0.952368
	Precisions for each class:
          A         B         C         D         E     F    G         H
0  0.985714  0.968571  0.994118  0.997685  0.894286  0.89  0.9  0.988571

	Recalls for each class:
     

For patient SMA54F:
	Balanced_accuracy:0.935086
	Precisions for each class:
          A         B      C         D         E         F         G         H
0  0.996764  0.897833  0.875  0.820896  0.988658  0.950769  0.996923  0.953846

	Recalls for each class:
          A         B         C         D         E         F         G  \
0  0.927711  0.986395  0.777778  0.916667  0.932264  0.968652  0.977376   

          H  
0  0.985692  

	Confusion matrix:
     A    B  C   D    E    F    G    H
0  308    0  0   1    0    0    0    0
1   23  290  1   3    6    0    0    0
2    0    0  7   0    1    0    0    0
3    0    0  0  55   12    0    0    0
4    0    2  1   0  523    0    0    3
5    0    1  0   1   15  618    9    6
6    1    1  0   0    0    0  648    0
7    0    0  0   0    4   20    6  620



For patient SMA60F:
	Balanced_accuracy:0.939225
	Precisions for each class:
          A         B         C        D     E     F         G         H
0  0.983957  0.995536  0.977011  0.922

## Learning curve

In [3]:
curve=pd.read_csv('./Galaxy13-[Model_Validation_on_data_9_and_data_2].tabular', sep='\t')
curve

,train_sizes_abs,mean_train_scores,std_train_scores,mean_test_scores,std_test_scores
0,5245,0.987132,0.001083,0.934450,0.003907
1,10490,0.972411,0.001365,0.940560,0.002551
2,15735,0.966202,0.001188,0.943255,0.002345
3,20980,0.961714,0.001025,0.945330,0.002479
4,26226,0.959473,0.000769,0.945588,0.002780
5,31471,0.957801,0.000587,0.946213,0.002760
6,36716,0.956785,0.000796,0.946308,0.003346
7,41961,0.955932,0.000885,0.946756,0.003151
8,47206,0.954988,0.000684,0.946608,0.002943
9,52452,0.954518,0.000346,0.946757,0.002461


In [4]:

data1 = go.Scatter(
        x = curve['train_sizes_abs'],
        y = curve['mean_train_scores'],
        mode = "lines",
        name = "Train scores",
)
data2 = go.Scatter(
    x = curve['train_sizes_abs'],
    y = curve['mean_test_scores'],
    mode = "lines",
    name="Test scores"
)
layout = dict(
    xaxis=dict(
        title='No. of samples'
    ),
    yaxis=dict(
        title='Balanced Accuracy'
    )
)
fig = go.Figure(data=[data1, data2], layout=layout)
iplot(fig)